In [ ]:
#!pip install gpxpy matplotlib numpy scipy
# !pip install fitparse pandas tqdm

In [12]:
from pathlib import Path
import io, gzip
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [8]:
DATA_DIR = Path("/Users/dana/Documents/code/Python/30DayMapChallenge/data/04_mydata") 
gpx_files = list(DATA_DIR.glob("*.fit.gz"))
print(f"Found {len(gpx_files)} files")

Found 671 files


In [10]:
# ----------------------------
# Helpers
# ----------------------------
def semicircles_to_degrees(val):
    return float(val) * (180.0 / (2**31)) if val is not None else None

def parse_fit_gz(path: Path):
    """Parse a .fit.gz file and return (lat, lon) arrays"""
    import io, gzip
    from fitparse import FitFile   

    with gzip.open(path, "rb") as f:
        data = f.read()
    ff = FitFile(io.BytesIO(data))
    lats, lons = [], []
    for rec in ff.get_messages("record"):
        lat = lon = None
        for f in rec:
            if f.name == "position_lat":
                lat = semicircles_to_degrees(f.value)
            elif f.name == "position_long":
                lon = semicircles_to_degrees(f.value)
        if lat is not None and lon is not None:
            lats.append(lat); lons.append(lon)
    return np.array(lats, dtype=float), np.array(lons, dtype=float)

In [13]:
all_lat, all_lon = [], []
skipped = 0
for p in tqdm(gpx_files, desc="Parsing FIT files"):
    try:
        lat, lon = parse_fit_gz(p)
        if lat.size:
            all_lat.append(lat)
            all_lon.append(lon)
    except Exception as e:
        skipped += 1
        print(f"Skipping {p.name}: {e}")

if not all_lat:
    raise SystemExit("No coordinates parsed from .fit.gz files!")

lats = np.concatenate(all_lat)
lons = np.concatenate(all_lon)

print(f"Parsed {lats.size:,} points from {len(gpx_files) - skipped} files (skipped {skipped})")
print(f"Extent: Lon {lons.min():.5f} .. {lons.max():.5f}, Lat {lats.min():.5f} .. {lats.max():.5f}")


Parsing FIT files: 100%|██████████| 671/671 [03:17<00:00,  3.40it/s]

Parsed 1,004,699 points from 671 files (skipped 0)
Extent: Lon -17.13534 .. 104.20634, Lat 2.77609 .. 52.24928
